In [16]:
%reload_ext autoreload
%autoreload 2

from transformers import BertConfig, BertModel
import torch

import sys 
sys.path.append('../')

from train import get_data, get_opt, construct_hyper_param
from sqlova.utils.utils_wikisql import get_models, get_fields, get_g, get_g_wvi_corenlp, get_wemb_bert, \
                                       get_g_wvi_bert_from_g_wvi_corenlp
import argparse

In [2]:
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

In [3]:
parser = argparse.ArgumentParser()
args = construct_hyper_param(parser, notebook=True)
args.bS = 8

BERT-type: uncased_L-12_H-768_A-12


In [19]:
path_h = '../data/sqlova_data' 
path_wikisql = '../data/sqlova_data'  
BERT_PT_PATH = path_wikisql

# Load Data

In [20]:
train_data, train_table, dev_data, dev_table, train_loader, dev_loader = get_data(path_wikisql, args)

# Load Model

In [6]:
path_model_bert = '../model_bert_best.pt'
path_model = '../model_best.pt'

In [7]:
# To start from the pre-trained models, un-comment following lines.

model, model_bert, tokenizer, bert_config = get_models(args, BERT_PT_PATH, trained=True,
                                                       path_model_bert=path_model_bert, path_model=path_model,
                                                       device=device)

Batch_size = 8
BERT parameters:
learning rate: 1e-05
Fine-tune BERT: False
Seq-to-SQL: the number of final BERT layers to be used: 2
Seq-to-SQL: the size of hidden dimension = 100
Seq-to-SQL: LSTM encoding layer size = 2
Seq-to-SQL: dropout rate = 0.3
Seq-to-SQL: learning rate = 0.001


In [8]:
opt, opt_bert = get_opt(args, model, model_bert, args.fine_tune)

# Train example

In [21]:
t = next(iter(train_loader))

In [10]:
nlu, nlu_t, sql_i, sql_q, sql_t, tb, hs_t, hds = get_fields(t, train_table, no_hs_t=True, no_sql_t=True)

In [22]:
g_sc, g_sa, g_wn, g_wc, g_wo, g_wv = get_g(sql_i)
g_wvi_corenlp = get_g_wvi_corenlp(t)

In [12]:
wemb_n, wemb_h, l_n, l_hpu, l_hs, \
nlu_tt, t_to_tt_idx, tt_to_t_idx  = get_wemb_bert(bert_config, 
                                                  model_bert, 
                                                  tokenizer, 
                                                  nlu_t, 
                                                  hds, 
                                                  args.max_seq_length,
                                                  num_out_layers_n=args.num_target_layers, 
                                                  num_out_layers_h=args.num_target_layers, 
                                                  device=device)

In [18]:
g_wvi_corenlp

[[[6, 7], [13, 15]],
 [[7, 8]],
 [[2, 2], [6, 6]],
 None,
 None,
 [[0, 0], [7, 8], [4, 4]],
 [[0, 1]],
 [[2, 2]]]

In [23]:
g_wvi_corenlp

[[[10, 12]],
 [],
 [[5, 11]],
 [[6, 8]],
 [[5, 6]],
 [[11, 12]],
 [[9, 10]],
 [[5, 5]]]

In [17]:
g_wvi = get_g_wvi_bert_from_g_wvi_corenlp(t_to_tt_idx, g_wvi_corenlp)

TypeError: 'NoneType' object is not iterable

In [13]:
wemb_n.shape

torch.Size([8, 32, 1536])